In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
experiments = client.search_experiments() #search_experiments() not list_experiments() 

In [2]:
for experiment in experiments:
    print("Experiment ID: {}, Name: {}".format(experiment.experiment_id, experiment.name))

Experiment ID: 2, Name: my-cool-experiment
Experiment ID: 1, Name: nyc-taxi-duration-prediction-experiment
Experiment ID: 0, Name: Default


In [3]:
# client.create_experiment(name="my-cool-experiment")

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.mse_ridge > 5",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.mse_ridge ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, mse: {run.data.metrics['mse_ridge']:.4f}")

run id: 68fd832dbbac4fb0bec9cd416b9f9b12, mse: 7.9611
run id: aaf50503c8024fe688da22d1640e2b6e, mse: 7.9611
run id: b6fc2de739c04b4bb936cbdfff80a106, mse: 7.9611
run id: 1e0fb4e68e6c4d7083520002b9ff3b07, mse: 7.9611
run id: 84d04d6faf34461190bdc9ae0ab1b40f, mse: 7.9611


In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id = "b6fc2de739c04b4bb936cbdfff80a106"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1704840655585, current_stage='None', description=None, last_updated_timestamp=1704840655585, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link=None, source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/b6fc2de739c04b4bb936cbdfff80a106/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [16]:
client.search_registered_models() #search_registered_models() not list_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1704839432557, description=('Travel duration predictor models of NYC green taxi using regressors such as '
  'linear, lasso and ridge regression.\n'), last_updated_timestamp=1704840655585, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1704840655585, current_stage='None', description=None, last_updated_timestamp=1704840655585, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link=None, source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/b6fc2de739c04b4bb936cbdfff80a106/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
  <ModelVersion: aliases=[], creation_timestamp=1704840092717, current_stage='Staging', description=None, last_updated_timestamp=1704840631416, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link=None, source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/b6fc2de739c04b4bb936cbdfff80a106/artifacts

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name) 

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None
version: 2, stage: Staging


C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_11436\3094455033.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [18]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False
)

C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_11436\3248838605.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1704840655585, current_stage='Staging', description=None, last_updated_timestamp=1704840974096, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link=None, source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/b6fc2de739c04b4bb936cbdfff80a106/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
from datetime import datetime

date = datetime.today().date()
model_version = 3
new_stage = "Staging"

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)


<ModelVersion: aliases=[], creation_timestamp=1704840655585, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-01-09', last_updated_timestamp=1704843589240, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link=None, source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/b6fc2de739c04b4bb936cbdfff80a106/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [25]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [26]:
df = read_dataframe('D:\RGT\Code\MLOps\data\green_tripdata_2021-07.csv')

C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_11436\3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [28]:
client.download_artifacts(run_id="86cefba2ac7345a29c36df05a963c5f1", path='preprocessor', dst_path='.')

'd:\\RGT\\Code\\MLOps\\Experiment-tracking\\preprocessor'

In [29]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [30]:
X_test = preprocess(df, dv)

In [31]:
target = "duration"
y_test = df[target].values

In [34]:
%time test_model(name="nyc-taxi-xgboost", stage="Staging", X_test=X_test, y_test=y_test)

d:\RGT\Code\MLOps\mlopsenv\lib\site-packages\xgboost\core.py:160: UserWarning: [15:55:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 25.3 s
Wall time: 5.86 s


{'rmse': 7.06766949262067}

In [35]:
client.transition_model_version_stage(
    name="nyc-taxi-xgboost",
    version=1,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_11436\2814322798.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1704836503792, current_stage='Production', description='', last_updated_timestamp=1704844745307, name='nyc-taxi-xgboost', run_id='86cefba2ac7345a29c36df05a963c5f1', run_link='', source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/86cefba2ac7345a29c36df05a963c5f1/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [36]:
%time test_model(name="nyc-taxi-xgboost", stage="Production", X_test=X_test, y_test=y_test)

d:\RGT\Code\MLOps\mlopsenv\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
d:\RGT\Code\MLOps\mlopsenv\lib\site-packages\xgboost\core.py:160: UserWarning: [15:59:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 25.2 s
Wall time: 4.41 s


{'rmse': 7.06766949262067}